In [ ]:
### 랜덤 포레스트 (Random Forest)
# 집단 학습을 기반으로 고정밀 분류, 회귀, 클러스터링 등을 구현
# 학습 전용 데이터를 기반으로 다수의 의사결정 트리를 만들고, 트리를 기반으로 다수결로 결과를 유도 -> 높은 정밀도
# 학습 데이터를 무작위로 샘플링해서 만들어진 다수의 의사결정 트리를 사용 => 랜덤 포레스트
# 의사결정 트리 = 트리 구조로 되어있는 그래프 = 예측과 분류를 수행하는 알고리즘 자체

In [2]:
### 랜덤 포레스트 사용하기  - 버섯 데이터 가져오기

import urllib.request as req

local = 'data/mushroom.csv'
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
req.urlretrieve(url, local)

('data/mushroom.csv', <http.client.HTTPMessage at 0x2d5b5f3f070>)

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

### 데이터 읽기
mr = pd.read_csv('data/mushroom.csv', header=None)

### 문자인 데이터를 숫자로 변경
label = []
data = []

for row_index, row in mr.iterrows() :   # 한 행의 데이터를 뽑는 함수 - 결국 row_index는 열 번호를 말하게 됨!
    label.append(row.iloc[0])   # 한 행에서 0번째 값 - p / e 값을 추출 = 정답만 추출
    row_data = []
    for val in row.iloc[1:] :
        row_data.append(ord(val)) # ord() : 문자를 숫자로 변환
    data.append(row_data)

### 학습 전용과 테스트 전용 데이터로 나누기 
data_train, data_test, label_train, label_test = train_test_split(data, label)

### 데이터 학습시키기
clf = RandomForestClassifier()
clf.fit(data_train, label_train)

### 데이터 예측하기
predict = clf.predict(data_test)

### 결과 테스트하기
ac_score = metrics.accuracy_score(label_test, predict)  # 예측값과 결과값 대조하여 정답률 산출
cl_report = metrics.classification_report(label_test, predict)
print('정답률 =', ac_score)
print('리포트 =', cl_report)

정답률 = 1.0
리포트 =               precision    recall  f1-score   support

           e       1.00      1.00      1.00      1061
           p       1.00      1.00      1.00       970

    accuracy                           1.00      2031
   macro avg       1.00      1.00      1.00      2031
weighted avg       1.00      1.00      1.00      2031



In [ ]:
### 랜덤포레스트 - 데이터 내부의 분류변수로 전개하기
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

### 데이터 읽기
mr = pd.read_csv('data/mushroom.csv', header=None)

### 문자인 데이터를 숫자로 변경
label = []
data = []
attr_list = []      # 보유하고있는 속성이 같을 경우 그룹화해주기 위한 기준을 만드는 리스트

for row_index, row in mr.iterrows() :   # 한 행의 데이터를 뽑는 함수 - 결국 row_index는 열 번호를 말하게 됨!
    label.append(row.iloc[0])   # 한 행에서 0번째 값 - p / e 값을 추출 = 정답만 추출
    exdata = []
    for col, val in enumerate(row.iloc[1:]) :   # 한 행에서 각 열 값을 하나씩 추출
        
        if row_index == 0 : # 첫 값은 정답이기 때문에 넣지 않기 & 한 행마다 각각의 열 값 분류를 위한 딕셔너리 만들기
            attr = {'dic' : {}, 'cnt' : 0}  # {'dic' : {속성 : idx}, 'cnt' : 해당 값을 가진 행 개수}
            attr_list.append(attr)
        else :
            attr = attr_list[col]       # 같은 열위치의 딕셔너리에 값 추가를 위해 attr에 저장

        # 버섯의 특정 기호를 배열로 나타내기
        d = [0,0,0,0,0,0,0,0,0,0,0,0]   # 속성데이터가 최대 12개이기 때문에 12개의 배열 생성
        if val in attr['dic'] :     # 속성값이 해당 열 위치의 딕셔너리에 존재할 경우 idx에 해당 속성의 값 저장
            idx = attr['dic'][val]
        else :
            idx = attr['cnt']       # 속성값이 해당 열 위치의 딕셔너리에 존재하지 않을 경우 idx에 cnt값을 저장
            attr['dic'][val] = idx  # 
            attr['cnt'] += 1
        d[idx] = 1
        exdata += d
    data.append(exdata)
    
print(attr_list)
#print(data)
            


# ### 학습 전용과 테스트 전용 데이터로 나누기 
# data_train, data_test, label_train, label_test = train_test_split(data, label)

# ### 데이터 학습시키기
# clf = RandomForestClassifier()
# clf.fit(data_train, label_train)

# ### 데이터 예측하기
# predict = clf.predict(data_test)

# ### 결과 테스트하기
# ac_score = metrics.accuracy_score(label_test, predict)  # 예측값과 결과값 대조하여 정답률 산출
# cl_report = metrics.classification_report(label_test, predict)
# print('정답률 =', ac_score)
# print('리포트 =', cl_report)

#### Cross-Validation - K 분할 교차 검증

In [ ]:
### 데이터를 검증하는 방법
## 크로스 밸리데이션(Cross-Validation) - 교차검증
# 특정 데이터를 훈련 전용 데이터와 테스트 전용 데이터로 분류 후 교차해서 사용 및 학습하여 타당성 검증

### K 분할 교차 검증(K-fold cross validation)

# 집합 X를 3개로 분할 -> A/B/C로 만드는 경우
# A를 테스트 전용 데이터, 나머지 B/C를 훈련 전용 데이터로 사용 -> 분류 정밀도 s1
# B를 테스트 전용 데이터, 나머지 A/C를 훈련 전용 데이터로 사용 -> 분류 정밀도 s2
# C를 테스트 전용 데이터, 나머지 A/B를 훈련 전용 데이터로 사용 -> 분류 정밀도 s3
# =>  s1, s2, s3의 평균을 구해 최종적인 분류 정밀도 산출

In [38]:
### 크로스 밸리데이션 사용
from sklearn import svm, metrics
import random, re

## 붓꽃의 데이터 읽기
lines = open('data/iris.csv', 'r', encoding='utf-8').read().split('\n')
f_tonum = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
f_cols = lambda li : list(map(f_tonum, li.strip().split(',')))
csv = list(map(f_cols, lines))
del csv[0]
random.shuffle(csv)

## 데이터를 K개로 분할
K = 5
csv_k = [[] for i in range(K)]      # 분할 개수만큼의 리스트 생성
for i in range(len(csv)) :
    csv_k[i % K].append(csv[i])

## 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하는 함수
def split_data_label(rows) :
    data = []; label = []
    for row in rows :
        data.append(row[0:4])
        label.append(row[4])
    return (data, label)

## 정답률 구하는 함수
def calc_score(test, train) :
    test_f, test_l = split_data_label(test)
    train_f, train_l = split_data_label(train)
    
    # 학습시키고 정답률 구하기
    clf = svm.SVC()
    clf.fit(train_f, train_l)
    pre = clf.predict(test_f)
    return metrics.accuracy_score(test_l, pre)

# K개로 분할해서 정답률 구하기
score_list = []
for testc in csv_k :
    # testc 이외에 데이터를 훈련용으로 넣기
    trainc = []
    for i in csv_k :
        if i != testc : trainc += i
    sc = calc_score(testc, trainc)
    score_list.append(sc)

print('각각의 정답률 :', score_list)
print('평균 정답률 :', sum(score_list)/len(score_list))
    

각각의 정답률 : [0.9333333333333333, 0.9666666666666667, 0.9, 1.0, 1.0]
평균 정답률 : 0.96


In [42]:
### scikit-learn의 크로스 밸리데이션 사용 : 메서드를 활용 (model_selection.cross_val_score())
import pandas as pd
from sklearn import svm, metrics, model_selection
import random, re

csv = pd.read_csv('data/iris.csv')

# DF를 훈련 전용 데이터와 테스트 전용 데이터로 분할
data = csv[['SepalLength','SepalWidth','PetalLength','PetalWidth']]
label = csv['Name']

# 크로스 밸리데이션
clf = svm.SVC()
scores = model_selection.cross_val_score(clf, data, label, cv=5)
print('각각의 정답률 :', scores)
print('평균 정답률 :', scores.mean())

각각의 정답률 : [0.96666667 0.96666667 0.96666667 0.93333333 1.        ]
평균 정답률 : 0.9666666666666666


#### 그리드 서치 

In [ ]:
### 그리드서치 : 어떤 매개변수가 적절한지 자동으로 조사
#               각 매개변수를 적당한 범위 내부에서 변경하면서 가장 성능이 좋을 때의 값을 찾는 방법
#               model_selection에서 그리드 서치를 위한 GirdSearchCV() 메서드 제공

In [28]:
import pandas as pd
import sklearn
from sklearn import model_selection, svm, metrics


# 손글씨 학습데이터 읽어오기        - 학습데이터 1000개 추출 당시 정답률 0.88정도
train_csv = pd.read_csv('data/mnist/train.csv')
test_csv = pd.read_csv('data/mnist/t10k.csv')

# 필요한 열 추출하기 iloc[행, 열]
train_label = train_csv.iloc[:, 0]
train_data = train_csv.iloc[:, 1:577]
test_label = test_csv.iloc[:,0]
test_data = test_csv.iloc[:,1:577]
print('학습 데이터 수 :', len(train_label))

# 그리드 서치 매개변수 설정 (여러 케이스 넣어서 그 중 최적조건 도출)
params = [
    {'C' : [1, 10, 100, 1000], 'kernel' : ['linear']},
    {'C' : [1, 10, 100, 1000], 'kernel':['rbf']},
    {'C' : [1, 10, 100, 1000], 'kernel':['poly'], 'gamma' : [0.001, 0.0001]},
    {'C' : [1, 10, 100, 1000], 'kernel':['rbf'], 'gamma' : [0.001, 0.0001]}  
]   

# 그리드 서치 수행
clf = model_selection.GridSearchCV(svm.SVC(), params, n_jobs= -1)
            # n_jobs = -1 means using all processors = 자동지정 (병렬 계산 프로세스 수 지정)
clf.fit(train_data, train_label)
print('학습기 =', clf.best_estimator_)

# 테스트 데이터 확인하기
predict = clf.predict(test_data)
ac_score = metrics.accuracy_score(test_label, predict)
print('정답률 =', ac_score)
print('sklearn :', sklearn.__version__)

학습 데이터 수 : 1000
학습기 = SVC(C=10)
정답률 = 0.904
sklearn : 1.1.1


C:\Users\tjoeun709\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0.451
- 0.452
- 0.453
- 0.454
- 0.455
- ...
Feature names seen at fit time, yet now missing:
- 1
- 1.1
- 107
- 108
- 11
- ...

  warnings.warn(message, FutureWarning)
